Given is a pair of vectors, and a comparison between vector and loop based implementation.

In [1]:
import numpy as np

In [2]:
a = [i + 1 for i in range(0, 500)]
b = [i for i in range(0, 500)]
dist_squared = sum([(a_i - b_i)**2 for a_i, b_i in zip(a, b)])
dist_squared

500

In [3]:
a_numpy = np.array(a)
b_numpy = np.array(b)
dist_squared = np.sum(np.square(a_numpy - b_numpy))
dist_squared

np.int64(500)

In [4]:
# using pure python
%timeit dist_squared = sum([(a_i - b_i)**2 for a_i, b_i in zip(a, b)])

39.4 μs ± 428 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
# using numpy
%timeit dist_squared = np.sum(np.square(a_numpy - b_numpy))

14.4 μs ± 4.53 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Now we want to calculate the euclidian distance between not only two vectors, but between 2 sets of vectors, each represented as a matrix. Your implementations should use just numpy (do not use scipy, sklearn, pandas, etc).

1. Implement the loop based code.
2. Implement the vector based code, without broadcast
3. Same, using broadcast.
4. Compare the run time between the loop and vector based implementations (questions 1 and 2).
5. Calculate the l2 norm of a matrix with row-based vectors. You can not use np.linalg.norm.
6. Calculate the l1 norm of a matrix with row-based vectors. You can not use np.linalg.norm.

## 1. Loop-based code

In [9]:
import random
import math

NUM_VECTORS = 500

random.seed(42)

# lambda func for calculating distance
distance = lambda a, b : math.sqrt((b[0] - a[0]) ** 2 + (b[1] - a[1]) ** 2)

a = [[random.random(), random.random()] for _ in range(NUM_VECTORS)]
b = [[random.random(), random.random()] for _ in range(NUM_VECTORS)]

%timeit matrix = [[distance(a_i, b_i) for b_i in b] for a_i in a]

85.7 ms ± 512 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 2. Vector-based code without broadcast

Not sure if this is the intended solution, but we know:
$$
\| \mathbf{a} - \mathbf{b} \| ^ 2 = (\mathbf{a} - \mathbf{b}) \cdot (\mathbf{a} - \mathbf{b}) = (\mathbf{a} - \mathbf{b})^T(\mathbf{a} - \mathbf{b})
$$

Solving down, we get
$$
(\mathbf{a} ^ T - \mathbf{b} ^ T)(\mathbf{a} - \mathbf{b}) \\ 
= \mathbf{a}^T\mathbf{a} - 2\mathbf{b}^T\mathbf{a} + \mathbf{b}^T\mathbf{b} \\
= \| \mathbf{a} ^ 2 \| - 2\mathbf{b}^T\mathbf{a} + \| \mathbf{b} ^ 2 \|
$$



In [10]:
np.random.seed(42)

a_numpy = np.random.rand(500, 2)
b_numpy = np.random.rand(500, 2)

In [11]:
%%timeit

# without broadcasting
a_sq = np.sum(a_numpy**2, axis=1, keepdims=True)
b_sq = np.sum(b_numpy**2, axis=1, keepdims=True)

# manual broadcasting
a_sq_tiled = np.tile(a_sq, (1, 500))
b_sq_tiled = np.tile(b_sq, (1, 500))

# center term
BA_dot = (b_sq @ a_sq.T)

# algebra
sq_dist = a_sq_tiled - (2 * BA_dot) + b_sq_tiled
# clamp negatives to 0
sq_dist = np.maximum(sq_dist, 0)

dist = np.sqrt(sq_dist)
dist


7.52 ms ± 98 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 3. Using Broadcast

In [13]:
%%timeit

# without broadcasting
a_sq = np.sum(a_numpy**2, axis=1, keepdims=True)
b_sq = np.sum(b_numpy**2, axis=1, keepdims=True)

# center term
BA_dot = (b_sq @ a_sq.T)

# automatic broadcasting
sq_dist = a_sq - (2 * BA_dot) + b_sq
# clamp negatives to 0
sq_dist = np.maximum(sq_dist, 0)

dist = np.sqrt(sq_dist)
dist


5.28 ms ± 103 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Alternatively, we broadcast to 500 x 500 x 2 matrix first.

In [14]:
%%timeit

# broadcast first, tiling along x and y axes
diff = a_numpy[:, None, :] - b_numpy[None, :, :]

dist_sq = np.sum(diff ** 2, axis=2)

dist = np.sqrt(dist_sq)

11 ms ± 89.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 4. Comparison
By far, the slowest implementation of this process of calculating a distance matrix for a dataset $\{ \mathbf{v}_i \in \mathbb{R} ^ 2 | i=1, 2,...,500 \}$ was the usage of normal loops. The deviation of 85.7 ms $\pm$ 512 $\mu\text{s}$ is around 10-20 times longer than any other method we used. The fastest method was to simply do everything as a matrix operation, allowing numpy to auto-broadcast the rows and columns. A broadcasting, non-matrix solution was also efficient, albeit slower.

## 5. L2 norm of matrix
Here, we want to find the largest singular values $\sigma$ of `a_numpy`:

In [15]:
singular_vals = np.linalg.eigvals(a_numpy.T @ a_numpy)
sq_l2_norm = np.max(singular_vals)
l2_norm = np.sqrt(sq_l2_norm)

print(f"L2 norm: {l2_norm}")

# check with np.linalg.norm
np.linalg.norm(a_numpy, ord=2)

L2 norm: 16.85599702842614


np.float64(16.855997028426142)

## 6. L1 norm of matrix

In [16]:
l1_norm = np.max(np.sum(np.abs(a_numpy), axis=0))

print(f"L1 norm: {l1_norm}")

# check with np.linalg.norm
np.linalg.norm(a_numpy, ord=1)

L1 norm: 256.5233384754257


np.float64(256.5233384754257)